In [1]:
import pandas as pd
import plotly.express as px

In [2]:
pd.read_pickle("failed_experiments.pkl").columns

Index([], dtype='object')

In [17]:
df = pd.read_pickle("results.pkl")
# df = pd.concat([pd.read_pickle("Multiple Pairs Results.pkl"), pd.read_pickle("Single Pairs Results.pkl")])
df["nodes"] = df["nodes"].astype(str)
df["experiment_type"] = df["experiment_type"].map(lambda x: "Single Pairs" if x == "Single" else x)
df["perturber_class"] = df["perturber_class"].map(lambda x: "GreedyMin" if x == "MinFirst" else x)
sorted(df.columns)

['Add Times',
 'Final Distance',
 'Global Budget Slack',
 'IIS_edges',
 'IIS_global_budget',
 'IIS_paths',
 'Iterations',
 'LP Status',
 'Number of Edges',
 'Number of Paths',
 'Original Distance',
 'Perturb Times',
 'Perturbation Dict',
 'Perturbation Failure',
 'Status',
 'Success',
 'Supporting Paths',
 'Time Taken',
 'Total Perturbations',
 'condition_index',
 'configuration_index',
 'epsilon',
 'experiment_type',
 'global_budget',
 'graph_name',
 'k',
 'local_budget',
 'max_iterations',
 'max_paths',
 'min_path_length',
 'n_experiments',
 'n_nodes_per_experiment',
 'n_trials',
 'nodes',
 'pairs',
 'perturber_class',
 'source',
 'target',
 'top_k',
 'use_multithreading',
 'weights']

In [4]:
print(len(df), sum(df["Success"]), sum(df["Success"])/len(df))
px.histogram(df, x="perturber_class", color="Status",facet_col="experiment_type",hover_data=["LP Status"], category_orders={"perturber_class": ["PathAttack", "GreedyFirst", "GreedyMin"]}).show()

5400 4824 0.8933333333333333


In [5]:
df["Average Add Time"] = df["Add Times"].map(lambda x: sum(x) / len(x) if len(x)>0 else None)
df["Average Perturb Time"] = df["Perturb Times"].map(lambda x: sum(x) / len(x) if len(x)>0 else None)
df["Total Add Time"] = df["Add Times"].map(sum)
df["Total Perturb Time"] = df["Perturb Times"].map(sum)
df["Overhead Time"] = df["Time Taken"] - df["Total Add Time"] - df["Total Perturb Time"]

In [6]:
df.to_csv("results.csv")

In [7]:
index_cols = ["nodes", "condition_index", "configuration_index"]
global_cols = ["epsilon", "n_nodes_per_experiment", "n_experiments", "n_trials", "min_path_length", "use_multithreading", "global_budget", "local_budget"]
extra_cols = ["Status","Add Times", "Perturb Times", "Perturbation Dict","source", "target", "LP Status", "IIS_paths", "IIS_edges", "IIS_global_budget", "Supporting Paths", "Global Budget Slack"]
config_cols = ["perturber_class", "k", "graph_name", "weights", "experiment_type"]
data_cols = ["Time Taken", "Iterations", "Number of Paths", "Number of Edges", "Original Distance", "Final Distance", "Success", "Total Perturbations","Total Add Time", "Total Perturb Time", "Overhead Time"]

In [8]:
# Average Over Trials
a = df[index_cols+data_cols].groupby(index_cols).mean()
a = a.join(df.set_index(index_cols)[config_cols]) # Add back config information
a = a.drop_duplicates()
len(a)

1800

In [ ]:
for experiment_type in a["experiment_type"].unique():
    

In [9]:
## TODO: Instances where the LP is infeasible or runs out of iterations should be removed from the data

for y_val in ["Time Taken", "Total Perturbations", "Number of Paths"]:
    for experiment_type in a["experiment_type"].unique():
        if a.empty: continue
        fig = px.box(a[a["experiment_type"] == experiment_type], 
                    title=f"{experiment_type}: {y_val}",
                    # x="nodes", 
                    y=y_val, 
                    boxmode="group", 
                    color="perturber_class", 
                    facet_col="weights", 
                    facet_row="k",  
                    height=1000,
                    hover_data=data_cols,
                    category_orders={"Status": ["Success"]+[s for s in df["Status"].unique() if s != "Success"],
                                    "Success": [True, False]},
                    points="all",
                    # boxpoints=False,
                    )
        fig.update_xaxes(visible=False, matches=None)
        # fig.update_yaxes(range=[-1,13])
        # fig.update_traces(boxmean=True) 
        # fig.update_xaxes(visible=True)
        fig.show()

In [10]:
b = a.reset_index().set_index(["configuration_index", "condition_index"])

In [11]:

for experiment_type in a.experiment_type.unique():
    fig = px.bar(b[b["experiment_type"]==experiment_type], 
                title=f"{experiment_type} Results",
                x="nodes", 
                y="Time Taken", 
                barmode="group", 
                color="perturber_class", 
                facet_col="weights", 
                facet_row="k", 
                pattern_shape="Success", 
                # height=1000,
                # hover_data=["top_k", "Average Add Time", "Average Perturb Time", "Original Distance", "Number of Paths", "Number of Edges", "Iterations", 'Final Distance',"Status", "LP Status", "nodes"],
                category_orders={"experiment_type": ["Single", "Sets", "Multiple Pairs"], 
                                "Status": ["Success"]+[s for s in df["Status"].unique() if s != "Success"],
                                "Success": [True, False]}
                )
    fig.update_xaxes(visible=False)
    fig.update_xaxes(matches=None)
    # fig.update_xaxes(visible=True)
    fig.show()

In [12]:
len(a)

1800

In [23]:
c = a.reset_index().set_index(["condition_index", "nodes"]+config_cols[1:]).groupby("perturber_class")

In [42]:
optimal = c.get_group("PathAttack").reset_index().set_index(["condition_index", "nodes"]+config_cols[1:])
normalized_dfs = dict()
for y_val in ["Total Perturbations", "Time Taken"]:
    diffs = pd.DataFrame()
    for name, group in c:
        diffs[name] = (group.reset_index().set_index(["condition_index", "nodes"]+config_cols[1:])[y_val]/optimal[y_val])
    normalized_dfs[y_val] = diffs

In [61]:

normalized_dfs["Total Perturbations"].reset_index().groupby(["experiment_type", "graph_name", "weights", "k"]).mean()

condition_index  GreedyFirst  GreedyMin  \
experiment_type graph_name weights k                                            
Multiple Pairs  Facebook   Equal   3              5.0     9.416888   9.416888   
                                   5              5.0     9.318499   9.318499   
                           Poisson 3              3.0     8.135936  14.267743   
                                   5              3.0     7.713134  17.324330   
                           Uniform 3              1.0     7.081265   8.999803   
                                   5              1.0     6.738454  10.467219   
Single Pairs    Facebook   Equal   3              4.0     7.409731   7.409731   
                                   5              4.0     7.460002   7.460002   
                           Poisson 3              2.0    10.704807  19.122437   
                                   5              2.0    10.982023  26.594863   
                           Uniform 3              0.0     7.292454   6.055772   
                                   5              0.0     7.367894   7.129357   

                                      PathAttack  
experiment_type graph_name weights k              
Multiple Pairs  Facebook   Equal   3         1.0  
                                   5         1.0  
                           Poisson 3         1.0  
                                   5         1.0  
                           Uniform 3         1.0  
                                   5         1.0  
Single Pairs    Facebook   Equal   3         1.0  
                                   5         1.0  
                           Poisson 3         1.0  
                                   5         1.0  
                           Uniform 3         1.0  
                                   5         1.0

In [62]:
normalized_dfs["Time Taken"].reset_index().groupby(["experiment_type", "graph_name", "weights", "k"]).mean()

condition_index  GreedyFirst  GreedyMin  \
experiment_type graph_name weights k                                            
Multiple Pairs  Facebook   Equal   3              5.0     1.979066   1.981704   
                                   5              5.0     1.999896   2.002003   
                           Poisson 3              3.0     1.392944   3.709098   
                                   5              3.0     1.370042   3.863179   
                           Uniform 3              1.0     1.030569   2.030929   
                                   5              1.0     0.975304   1.988946   
Single Pairs    Facebook   Equal   3              4.0     0.905609   0.880352   
                                   5              4.0     0.903456   0.896044   
                           Poisson 3              2.0     1.205394   3.079266   
                                   5              2.0     1.188714   3.317361   
                           Uniform 3              0.0     0.913599   1.263249   
                                   5              0.0     0.961692   1.363487   

                                      PathAttack  
experiment_type graph_name weights k              
Multiple Pairs  Facebook   Equal   3         1.0  
                                   5         1.0  
                           Poisson 3         1.0  
                                   5         1.0  
                           Uniform 3         1.0  
                                   5         1.0  
Single Pairs    Facebook   Equal   3         1.0  
                                   5         1.0  
                           Poisson 3         1.0  
                                   5         1.0  
                           Uniform 3         1.0  
                                   5         1.0